In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 57.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12


In [ ]:
# Step 0: Confirm Drive is Mounted (Run this only once per session)
from google.colab import drive
import os

if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# Paths
model_path = '/content/drive/MyDrive/best_segmentation0.1v.pt'
video_path = '/content/drive/MyDrive/test_video.mp4'
output_path = '/content/drive/MyDrive/bubble_segmentation_output2_update_2.mp4'

# Verify files
for p in [model_path, video_path]:
    if not os.path.exists(p):
        raise FileNotFoundError(f"❌ File not found: {p}")
print("✅ Model & input video confirmed!")

Mounted at /content/drive
✅ Model & input video confirmed!


In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np

# Load model
model = YOLO(model_path)

# Open video
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise IOError("❌ Cannot open video file")

fps = cap.get(cv2.CAP_PROP_FPS) or 25
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Use 'mp4v' codec — most compatible with Colab + Drive
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

print(f"🎥 Video will be saved to:\n{output_path}")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
🎥 Video will be saved to:
/content/drive/MyDrive/bubble_segmentation_output2_update_2.mp4


In [ ]:
total_bubbles = 0
frame_number = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_number += 1

    # Run inference
    results = model.predict(source=[frame], conf=0.5, iou=0.4, verbose=False)

    frame_bubble_count = 0
    if results[0].masks is not None:
        masks = results[0].masks.xy
        frame_bubble_count = len(masks)
        total_bubbles += frame_bubble_count

        for mask_points in masks:
            mask_points = np.array(mask_points, dtype=np.int32)

            # Calculate area and radius
            area = cv2.contourArea(mask_points)
            radius_px = np.sqrt(area / np.pi) if area > 0 else 0

            # Find center of the mask (centroid)
            M = cv2.moments(mask_points)
            if M["m00"] != 0:
                cx = int(M["m10"] / M["m00"])
                cy = int(M["m01"] / M["m00"])
            else:
                cx = int(np.mean(mask_points[:, :, 0]))
                cy = int(np.mean(mask_points[:, :, 1]))

            # Draw green contour around bubble
            cv2.polylines(frame, [mask_points], True, (0, 255, 0), 2)

            # Draw radius label just above center
            cv2.putText(frame, f"R: {radius_px:.1f}px",
                        (cx - 20, cy - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    # Overlay dynamic info on top-left
    cv2.putText(frame, f"Frame: {frame_number}", (20, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    cv2.putText(frame, f"Current: {frame_bubble_count}", (20, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    cv2.putText(frame, f"Total: {total_bubbles}", (20, 90),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)

    # Write frame to output video
    out.write(frame)

# Release resources
cap.release()
out.release()
print(f"✅ Processing complete!")
print(f"💾 Video saved at:\n{output_path}")

✅ Processing complete!
💾 Video saved at:
/content/drive/MyDrive/bubble_segmentation_output2_update_2.mp4


In [ ]:
from IPython.display import HTML
from base64 import b64encode

# Read the saved video
with open(output_path, 'rb') as f:
    mp4 = f.read()

# Encode for display
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f'''
<video width=800 controls>
  <source src="{data_url}" type="video/mp4">
</video>
<p><b>▶️ Now playing: bubble_segmentation_output2_update_2.mp4</b></p>
''')

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
from ultralytics import YOLO

model_path = '/content/drive/MyDrive/best_segmentation0.1v.pt'
model = YOLO(model_path)

print("Model loaded successfully!")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Model loaded successfully!


In [ ]:
import cv2
import numpy as np

In [ ]:
video_path = '/content/drive/MyDrive/test_video.mp4'
cap = cv2.VideoCapture(video_path)

In [ ]:
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [ ]:
output_path = '/content/drive/MyDrive/bubble_segmentation_output_1.mp4'
out = cv2.VideoWriter(output_path,
                      cv2.VideoWriter_fourcc(*'mp4v'),
                      fps, (width, height))

total_bubbles = 0
frame_number = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_number += 1

    # ==== Step 3: YOLOv8 Segmentation Prediction ====
    results = model.predict(source=[frame], conf=0.5, iou=0.4, verbose=False)

    frame_bubble_count = 0
    if results[0].masks is not None:
        masks = results[0].masks.xy  #list of polygons
        frame_bubble_count = len(masks)
        total_bubbles += frame_bubble_count

        for mask_points in masks:
            mask_points = np.array(mask_points, dtype=np.int32)
            area = cv2.contourArea(mask_points)
            radius_px = np.sqrt(area / np.pi)  #Radius in pixels

            cv2.polylines(frame, [mask_points], isClosed=True, color=(0, 255, 0), thickness=2)

            x, y = mask_points[0]
            cv2.putText(frame, f"R:{radius_px:.1f}px", (int(x), int(y)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)

    # ==== Step 4: Overlay Info ====
    cv2.putText(frame, f"Frame: {frame_number}", (20, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    cv2.putText(frame, f"Frame Bubbles: {frame_bubble_count}", (20, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    cv2.putText(frame, f"Total Bubbles: {total_bubbles}", (20, 90),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)

    #Write to output
    out.write(frame)

cap.release()
out.release()

print(f"✅ Processing complete! Video saved at: {output_path}")

✅ Processing complete! Video saved at: /content/drive/MyDrive/bubble_segmentation_output_1.mp4
